<a href="https://colab.research.google.com/github/SaraWestWA/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/SW_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [2]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [3]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [4]:
# look at data
print(df.shape)
df.head()

(48817, 34)


,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
# list of column headers
col_list = list(df)
col_list

['bathrooms',
 'bedrooms',
 'created',
 'description',
 'display_address',
 'latitude',
 'longitude',
 'price',
 'street_address',
 'interest_level',
 'elevator',
 'cats_allowed',
 'hardwood_floors',
 'dogs_allowed',
 'doorman',
 'dishwasher',
 'no_fee',
 'laundry_in_building',
 'fitness_center',
 'pre-war',
 'laundry_in_unit',
 'roof_deck',
 'outdoor_space',
 'dining_room',
 'high_speed_internet',
 'balcony',
 'swimming_pool',
 'new_construction',
 'terrace',
 'exclusive',
 'loft',
 'garden_patio',
 'wheelchair_access',
 'common_outdoor_space',
 'outdoor_options']

In [33]:
# Identify popular features
df.iloc[:,10:34].sum(axis=0)

elevator                25621
cats_allowed            23348
hardwood_floors         23348
dogs_allowed            21852
doorman                 20740
dishwasher              20263
no_fee                  17920
laundry_in_building      2576
fitness_center          13105
pre-war                  9063
laundry_in_unit          8587
roof_deck                6481
outdoor_space            6756
dining_room              5020
high_speed_internet      4257
balcony                  2952
swimming_pool            2695
new_construction         2534
terrace                  2255
exclusive                2114
loft                     2085
garden_patio             1920
wheelchair_access        1329
common_outdoor_space     1281
dtype: int64

In [47]:
# Does the apartment have an option for being outdoors?
# add an outdoor options column
df['outdoor_options'] = 0
print(df['outdoor_options'].value_counts())
df.head(1)

0    48817
Name: outdoor_options, dtype: int64


,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,outdoor_options
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
# create a column list of outdoor options
oo_list = (
 'roof_deck',
 'outdoor_space',
 'balcony',
 'terrace',
 'garden_patio',
 'common_outdoor_space')
oo_list

('roof_deck',
 'outdoor_space',
 'balcony',
 'terrace',
 'garden_patio',
 'common_outdoor_space')

In [87]:
# populate outdoor_options column
# this looks like a promising feature
df['outdoor_options'] = df.loc[:,oo_list].sum(axis=1)
df['outdoor_options'].value_counts()


0    36379
1     6486
2     3793
3     1267
4      691
5      198
6        3
Name: outdoor_options, dtype: int64

In [89]:
# we only want to know whether or not a unit has an
# outdoor option at this time, not how many
df.loc[df['outdoor_options'] > 0, 'outdoor_options'] = 1
df['outdoor_options'].value_counts()


0    36379
1    12438
Name: outdoor_options, dtype: int64

In [48]:
# Does the apartment have an dishwasher or laundry available on-site?
# create an easy_clean column
df['easy_clean'] = 0
df.head(1)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,outdoor_options,easy_clean
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
# See how many units have two laundry options available

# ec_list=('laundry_in_building','laundry_in_unit')
# df['easy_clean'] = df.loc[:,ec_list].sum(axis=1)
# df['easy_clean'].value_counts()
# overlap is negligible, 86, ignore

ec_list=('laundry_in_building','laundry_in_unit','dishwasher')
df['easy_clean'] = df.loc[:,ec_list].sum(axis=1)
df['easy_clean'].value_counts()

0    25522
1    15241
2     7977
3       77
Name: easy_clean, dtype: int64

In [53]:
# find datatype for created(date) column
df['created'].dtypes

dtype('<M8[ns]')

In [54]:
# change date created to datetime format for sorting
# type is unfamiliar, but seems to be a specific datetime type

df['created'] = pd.to_datetime(df.created)
df['created'].dtypes

dtype('<M8[ns]')

In [55]:
df['created'].sort_values()

5186    2016-04-01 22:12:41
7945    2016-04-01 22:56:00
6424    2016-04-01 22:57:15
7719    2016-04-01 23:26:07
1723    2016-04-02 00:48:13
                ...        
19943   2016-06-29 17:47:34
16801   2016-06-29 17:56:12
32633   2016-06-29 18:14:48
20560   2016-06-29 18:30:41
17743   2016-06-29 21:41:47
Name: created, Length: 48817, dtype: datetime64[ns]

In [56]:
# see format of timestamp to use correctly in split
df['created'][0]

Timestamp('2016-06-24 07:54:24')

In [132]:
# Test/Train Split
train = df[df['created']< '2016-06-01 00:00:00']
test = df[df['created']>= '2016-06-01 00:00:00']

# verify split
print(train.shape)
print(test.shape)

(31844, 36)
(16973, 36)


In [133]:
# import linear regression esitmator from Skikit-Learn

from sklearn.linear_model import LinearRegression

# instantiate

model=LinearRegression()

In [161]:
# create features matrix (X) and target vector (y)

features = ['bedrooms','bathrooms', 'outdoor_options','easy_clean']
target = ['price']

X_train = train[features]
y_train = train[target]

X_train.shape, y_train.shape

((31844, 4), (31844, 1))

In [162]:
# try with three features, worked with 2 on the last assignment
# fit model
model.fit (X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [168]:
# predict the price of a 3 bed, 1.5 bath unit with an
# outdoor option and two easy clean features

beds =3
baths =1.5
out = 1
clean = 2

x_test = [[beds,baths,out,clean]]

y_pred = model.predict(x_test)
y_pred

array([[5162.07806516]])

In [167]:
# "slope" of model line, coefficients and intercept
m = model.coef_
b = model.intercept_

m, b

(array([[ 370.55702188, 1985.3297332 ,  120.45720092,  269.66539221]]),
 array([412.62441439]))

In [171]:
#create my own function

def myfunction(beds,baths,out,clean):
  y_pred = model.predict([[beds,baths,out,clean]])
  esitmate = y_pred[0]
  beds_coefficient = model.coef_[0,0]
  bath_coefficient = model.coef_[0,1]
  out_coefficient = model.coef_[0,2]
  clean_coefficient = model.coef_[0,3]
  base = model.intercept_[0]

  
  print(f'The base price, which is the y-intercept, for an apartment is ${base:,.0f} per month.')
  print(f'The cost per bedroom is ${beds_coefficient:,.0f} per month.')
  print(f'The cost per bathroom is ${bath_coefficient:,.0f} per month.')
  print(f'The cost for having an outdoor option is ${out_coefficient:,.0f} per month.')
  print(f'The cost for having laundry and a dishwasher is ${clean_coefficient:,.0f} per month.')

  # print ('The estimated price for a',beds,'bedroom',baths,'apartment is',(f'${esitmate[0]:,.0f} per month.'))


myfunction(4,1.5,1,2)

The base price, which is the y-intercept, for an apartment is $413 per month.
The cost per bedroom is $371 per month.
The cost per bathroom is $1,985 per month.
The cost for having an outdoor option is $120 per month.
The cost for having laundry and a dishwasher is $270 per month.


In [150]:
# #create my own function

# def myfunction(baths,out):
#   y_pred = model.predict([[baths,out]])
#   esitmate = y_pred[0]
#   bath_coefficient = model.coef_[0,0]
#   out_coefficient = model.coef_[0,1]
#   # clean_coefficient = model.coef_[0,2]
#   base = model.intercept_[0]

  
#   print(f'The base price, which is the y-intercept, for an apartment is ${base:,.0f} per month.')
#   # print(f'The cost per bedroom is ${bed_coefficient:,.0f} per month.')
#   print(f'The cost per bathroom is ${bath_coefficient:,.0f} per month.')
#   print(f'The cost for having an outdoor option is ${out_coefficient:,.0f} per month.')
#   # print(f'The cost for having laundry and a dishwasher is ${clean_coefficient:,.0f} per month.')

#   # print ('The estimated price for a',beds,'bedroom',baths,'apartment is',(f'${esitmate[0]:,.0f} per month.'))


# myfunction(4,1)

The base price, which is the y-intercept, for an apartment is $477 per month.
The cost per bathroom is $2,505 per month.
The cost for having an outdoor option is $324 per month.
